Моделирование математического маятника различными численными методами

В этом скрипте мы реализуем и сравним три численных метода интегрирования:
- Метод Эйлера
- Метод Leapfrog (симплектический метод)
- Метод Рунге-Кутты 2 порядка

Мы сравним их по энергетической консервативности и точности на примере математического маятника.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import matplotlib.animation as animation
from IPython.display import HTML

# Физические параметры и начальные условия

In [3]:

# Физические параметры маятника
g = 9.81  # ускорение свободного падения, м/с²
L = 1.0   # длина маятника, м
m = 1.0   # масса грузика, кг

# Начальные условия
theta0 = pi/6  # начальный угол, рад (30 градусов)
omega0 = 0.0   # начальная угловая скорость, рад/с

# Параметры интегрирования
T = 20.0      # время моделирования, с
dt = 0.1     # шаг интегрирования, с

In [4]:
# Массив времени
t = np.arange(0, T, dt)
n_steps = len(t)

print(f"Число шагов: {n_steps}")
print(f"Время моделирования: {T} с")
print(f"Шаг интегрирования: {dt} с")
print(f"Начальный угол: {theta0:.3f} рад ({theta0*180/pi:.1f}°)")

Число шагов: 200
Время моделирования: 20.0 с
Шаг интегрирования: 0.1 с
Начальный угол: 0.524 рад (30.0°)


In [5]:
def energy(theta, omega):
    """
    Расчет полной энергии маятника

    Args:
        theta: угол отклонения, рад
        omega: угловая скорость, рад/с

    Returns:
        E: полная энергия, Дж
    """
    # Потенциальная энергия
    U = m * g * L * (1 - np.cos(theta))
    # Кинетическая энергия
    K = 0.5 * m * (L * omega)**2
    return U + K


In [6]:
def analytical_solution(theta0, omega0, dt, n_steps):
    """
    Аналитическое решение для маятника (приближение малых колебаний)

    Args:
        theta0: начальный угол, рад
        omega0: начальная угловая скорость, рад/с
        t: массив времени

    Returns:
        theta_analytical: угол отклонения
        omega_analytical: угловая скорость
    """

    omega_freq = np.sqrt(g / L)  # собственная частота

    y_state = np.zeros(n_steps, 3)
    
    y_state[0] = np.arange(0, n_steps*dt, dt) # время
    y_state[1] = theta0 * np.cos(omega_freq * t) + (omega0 / omega_freq) * np.sin(omega_freq * t) # угол
    y_state[2] = -theta0 * omega_freq * np.sin(omega_freq * t) + omega0 * np.cos(omega_freq * t) # угловая скорость

    return y_state


# Численные методы интегрирования

## Метод Рунге-Кутты 2 порядка

### Физическое описание системы

Для математического маятника мы имеем систему дифференциальных уравнений:

$$\frac{d\vec{Y}}{dt} = \vec{F}(\vec{Y})$$

где вектор состояния:
$$\vec{Y} = \begin{pmatrix}t \\ \theta \\ \omega \end{pmatrix}$$

а функция изменения:
$$\vec{F}(\vec{Y}) = \begin{pmatrix} 1 \\ \omega \\ -\frac{g}{L} \sin(\theta) \end{pmatrix}$$

### Метод средней точки (RK2)

Метод Рунге-Кутты второго порядка использует две оценки производной:

1. **Первая оценка (в точке n):**
   $$\vec{k}_1 = \vec{F}(\vec{Y}_n)$$

2. **Вторая оценка (в средней точке):**
   $$\vec{k}_2 = \vec{F}\left(\vec{Y}_n + \vec{k}_1 \cdot \frac{\Delta t}{2}\right)$$

3. **Итоговое обновление:**
   $$\vec{Y}_{n+1} = \vec{Y}_n + \frac{\vec{k}_1 + \vec{k}_2}{2} \cdot \Delta t$$

### В компонентной форме:

$$\vec{k}_1 = \begin{pmatrix}1 \\ \omega_n \\ -\frac{g}{L} \sin(\theta_n) \end{pmatrix}$$

$$\vec{k}_2 = \begin{pmatrix}1 \\ \omega_n + k_{1,\omega} \cdot \frac{\Delta t}{2} \\ -\frac{g}{L} \sin\left(\theta_n + k_{1,\theta} \cdot \frac{\Delta t}{2}\right) \end{pmatrix}$$

$$\vec{Y}_{n+1} = \vec{Y}_n + \frac{\vec{k}_1 + \vec{k}_2}{2} \cdot \Delta t$$


In [7]:
def func_f(y):
    """ Функция изменения принимаен на вход np.array(shape=3)"""
    t, theta, omega = y
    return [
        1,
        omega
        -(g / L) * np.sin(theta),
    ]


def rk2_method(theta0, omega0, dt, n_steps):
    """
    Метод Рунге-Кутты 2 порядка для маятника

    Args:
        theta0: начальный угол, рад
        omega0: начальная угловая скорость, рад/с
        dt: шаг интегрирования, с
        n_steps: число шагов

    Returns:
        theta: массив углов
        omega: массив угловых скоростей
    """
    y_state = np.zeros(n_steps, 3)

    y_state[0] = [0, theta0, omega0]

    for i in range(n_steps - 1):
        k1 = func_f(y_state[i])
        k2 = func_f(y_state[i]+k1*dt)
        y_state[i+1] = y_state[i] + (k1 + k2) * dt / 2

    return y_state

## Метод Leapfrog (симплектический метод чередования сетки)

### Концепция состояний системы

Метод Leapfrog основан на концепции **чередования сетки** с состояниями на целом и полуцелом шагах:

**Состояния на целом шаге (n, n+1, ...):**
- Угол отклонения θ_n
- Ускорение α_n = -(g/L)sin(θ_n)

**Состояния на полуцелом шаге (n+1/2, n+3/2, ...):**
- Угловая скорость ω_{n+1/2}

### Расширенное векторное представление

Для лучшего понимания введем два вектора состояния:

**Вектор положения (на целом шаге):**
$$\vec{Y}_n = \begin{pmatrix} t_n \\ \theta_n \end{pmatrix}$$

**Вектор скорости (на полуцелом шаге):**
$$\vec{V}_{n+1/2} = \begin{pmatrix} t_{n+1/2} \\ \omega_{n+1/2} \end{pmatrix}$$

### Алгоритм метода:

1. **Вычисление скорости на полуцелом шаге:**
   $$\vec{V}_{n+1/2} = \vec{V}_{n-1/2} + \Delta t \cdot \vec{F}(\vec{Y}_n)$$

2. **Вычисление положения на целом шаге:**
   $$\vec{Y}_{n+1} = \vec{Y}_n + \Delta t \cdot \vec{G}(\vec{V}_{n+1/2})$$


### Симплектические свойства

Метод Leapfrog является **симплектическим**, что означает:
- Отличное сохранение энергии консервативных систем
- Сохранение фазового объема
- Стабильность при долгосрочном моделировании
- Геометрическая точность в фазовом пространстве

In [8]:
def func_u(v):
    """ Функция изменения принимаен на вход np.array(shape=1)"""
    omega = v[0]
    return [
        1,
        omega
    ]

def func_v(u):
    """ Функция изменения принимаен на вход np.array(shape=2)"""
    t, theta = u
    return [
        -(g / L) * np.sin(theta)
    ]

def leapfrog_method(theta0, omega0, dt, n_steps):
    """
    Метод Leapfrog для маятника

    Args:
        theta0: начальный угол, рад
        omega0: начальная угловая скорость, рад/с
        dt: шаг интегрирования, с
        n_steps: число шагов

    Returns:
        theta: массив углов
        omega: массив угловых скоростей
    """

    u_state = np.zeros(n_steps, 2)
    v_state = np.zeros(n_steps, 1)
    v2_state = np.zeros(n_steps, 1)

    u_state[0] = [0, theta0]
    v_state[0] = [omega0]

    v2_state[0] = v_state[0] + func_v(u_state[0]) * dt / 2

    for i in range(n_steps - 1):
        # Шаг для положения (0 -> 1 -> 2 ...)
        u_state[i+1] = u_state[i] + func_u(v_state[i]) * dt
        # Шаг для скорости (0.5 -> 1.5 -> 2.5 ...)
        v2_state[i+1] = v2_state[i] + func_v(u_state[i+1]) * dt

    v_state[1:] = (v2_state[:-1] + v2_state[1:])/2

    return u_state, v_state